# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

Импортируем библиотеки, необходимые для работы.

In [1]:
import nltk
import numpy as np
import pandas as pd
import re
import string
import transformers
import warnings

from catboost import CatBoostClassifier
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer

from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.metrics import f1_score, classification_report
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.utils import shuffle
from tqdm.notebook import tqdm

tqdm.pandas()
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('wordnet')
stop_words = set(stopwords.words('english'))

# Отключим оповещения
pd.options.mode.chained_assignment = None
warnings.filterwarnings("ignore")

# Зададим константы, которые будем использовать во всём проекте.
RANDOM_STATE = 42

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\roum-\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\roum-\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\roum-\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## 1. Подготовка

### 1.1. Загрузим данные из файла

### 1.2. Ознакомимся с данными, посмотрим основную информацию.

In [3]:
toxic_comments.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [4]:
toxic_comments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


Пропущенных значений нет, типы данных соответствуют.

In [5]:
toxic_comments.duplicated().sum()

0

Дубликатов нет.

### 1.3. Подготовим данные.

In [6]:
len(toxic_comments['Unnamed: 0'].unique())

159292

В столбце 'Unnamed: 0' содержатся только уникальные значения и они дублируют индекс, поэтому удалим его.

In [7]:
toxic_comments.drop(columns=['Unnamed: 0'], inplace=True)

In [8]:
toxic_comments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


Ознакомимся с классами, которые предстоит предсказывать модели.

In [9]:
toxic_comments.toxic.value_counts()

0    143106
1     16186
Name: toxic, dtype: int64

Классы несбалансировааны, преобладает класс со значением 0.

In [10]:
toxic_comments.toxic.value_counts()/toxic_comments.shape[0]

0    0.898388
1    0.101612
Name: toxic, dtype: float64

Данные подготовлены.

### 1.4. Подготовим признаки

Воспользуемся функциями для очистки текста и его лемматизации.

In [11]:
def clear_text(text):
    text_clear = re.sub(r'[^a-zA-Z ]', ' ', text)
    list_clear = text_clear.split()
    text = " ".join(list_clear)
    return text

# Lemmatize with POS Tag
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    # Lemmatize a Sentence with the appropriate POS tag
    # lemm_list = [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(text) if w not in string.punctuation]
    lemm_list = [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(clear_text(text))]
    lemm_text = " ".join(lemm_list)
    return lemm_text


In [12]:
tqdm.pandas()

Выполним подготовку признаков.

In [13]:
%%time
# df_test['lemm_text'] = df_test.text.apply(lemmatize_text)
toxic_comments['lemm_text'] = toxic_comments.text.progress_apply(lemmatize_text)

  0%|          | 0/159292 [00:00<?, ?it/s]

CPU times: total: 14min 24s
Wall time: 31min 55s


### 1.5. Выделим целевой признак и разделим данные на обучающую и тестовую выборки.

In [14]:
features = toxic_comments.lemm_text
target = toxic_comments.toxic

Разобьём выборку на обучающую и тестовую.

In [15]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=.25, stratify=target)

In [16]:
print(f'X_train: {X_train.shape}')
print(f'X_test: {X_test.shape}')
print(f'y_train: {y_train.shape}')
print(f'y_test: {y_test.shape}')

X_train: (119469,)
X_test: (39823,)
y_train: (119469,)
y_test: (39823,)


## Вывод

Загрузили данные и проверили их.

Пропущенных значений и дубликатов нет.

Обнаружили столбец 'Unnamed: 0', который оказался лишним, его удалили.

Классы в данных оказались несбалансированными.

Полученную выборку использовали для подготовки признаков.

Текст комментариев был очищен от лишних символов и произведена его токенизация и лемматизация.

После этого данные были разделены на обучающую и тестовую выборки.

## 2. Обучение

Обучим модели и выберем лучшую.

### 2.1. LogisticRegression

In [17]:
%%time
vectorizer =  TfidfVectorizer(stop_words=stop_words)

# model_rf = RandomForestClassifier(random_state=RANDOM_STATE)

model_lr = LogisticRegression(random_state=RANDOM_STATE, n_jobs=-1, class_weight='balanced')

pipline_lr = make_pipeline(vectorizer, model_lr)

param_grid_lr = {
    'logisticregression__C': range(10, 25, 1),
    'logisticregression__max_iter': range(200, 501, 100)
}

# получается достаточно много комбинаций гиперпараметров при переборе
# будем использовать RandomizedSearchCV, он работает на много быстрее
gs_lr = RandomizedSearchCV(
    pipline_lr, 
    param_distributions=param_grid_lr, 
    scoring='f1', 
    n_jobs=-1,
    random_state=RANDOM_STATE
)

gs_lr.fit(X_train, y_train)

gs_lr_best_score = gs_lr.best_score_
gs_lr_best_params = gs_lr.best_params_
print(f'best_F1: {gs_lr_best_score}')
print(f'best_params: {gs_lr_best_params}')

best_F1: 0.7563296809250455
best_params: {'logisticregression__max_iter': 200, 'logisticregression__C': 10}
CPU times: total: 9.78 s
Wall time: 1min 24s


Модель показала значение метрики f1, соответствующей условиям задания.

### 2.2. RandomForest

In [18]:
%%time
vectorizer =  TfidfVectorizer(stop_words=stop_words)

# model_rf = RandomForestClassifier(random_state=RANDOM_STATE)

model_rf = RandomForestClassifier(random_state=RANDOM_STATE, n_jobs=-1, class_weight='balanced')

pipline_rf = make_pipeline(vectorizer, model_rf)

param_grid_rf = {
    'randomforestclassifier__n_estimators': range(10, 20, 2),
    'randomforestclassifier__max_depth': range(3, 11, 2),
    'randomforestclassifier__max_features': range(8,19,2),
    'randomforestclassifier__min_samples_split': (2, 3),
    'randomforestclassifier__min_samples_leaf': (1, 2, 3)
}


# получается достаточно много комбинаций гиперпараметров при переборе
# будем использовать RandomizedSearchCV, он работает на много быстрее
gs_rf = RandomizedSearchCV(
    pipline_rf, 
    param_distributions=param_grid_rf, 
    scoring='f1', 
    n_jobs=-1,
    random_state=RANDOM_STATE
)

gs_rf.fit(X_train, y_train)

gs_rf_best_score = gs_rf.best_score_
gs_rf_best_params = gs_rf.best_params_
print(f'best_F1: {gs_rf_best_score}')
print(f'best_params: {gs_rf_best_params}')

best_F1: 0.19441543738671724
best_params: {'randomforestclassifier__n_estimators': 10, 'randomforestclassifier__min_samples_split': 2, 'randomforestclassifier__min_samples_leaf': 2, 'randomforestclassifier__max_features': 18, 'randomforestclassifier__max_depth': 5}
CPU times: total: 7.33 s
Wall time: 28.3 s


Модель показала низкое значение метрики f1 и не подходит по условиям задания.

### 2.3. CatBoost

In [19]:
%%time
vectorizer =  TfidfVectorizer(stop_words=stop_words)

model_cb = CatBoostClassifier(random_state=RANDOM_STATE)

pipline_cb = make_pipeline(vectorizer, model_cb)

param_grid_cb = {'catboostclassifier__learning_rate':[x/10 for x in range(1, 3)], 
                 'catboostclassifier__iterations':[223], 
                 'catboostclassifier__verbose':[False]
                }

# получается достаточно много комбинаций гиперпараметров при переборе
# будем использовать RandomizedSearchCV, он работает на много быстрее
gs_cb = RandomizedSearchCV(
    pipline_cb,
    param_distributions=param_grid_cb,
    scoring='f1', 
    n_jobs=-1,
    cv = 3,
    random_state=RANDOM_STATE
)

gs_cb.fit(X_train, y_train)
# , eval_metric='rmse', categorical_feature=cat_features_ohe+cat_features_oe

gs_cb_best_score = gs_cb.best_score_
gs_cb_best_params = gs_cb.best_params_
print(f'best_F1: {gs_cb_best_score}')
print(f'best_params: {gs_cb_best_params}')

best_F1: nan
best_params: {'catboostclassifier__verbose': False, 'catboostclassifier__learning_rate': 0.1, 'catboostclassifier__iterations': 223}
CPU times: total: 10min 14s
Wall time: 4min 6s


Модель показала значение метрики f1, соответствующее условиям задания.

## Вывод

Наилучшие показатели метрики F1 показала модель LogisticRegression, проверим её на тестовой выборке.

## 3. Тестирование

In [20]:
vectorizer =  TfidfVectorizer(stop_words=stop_words)
X_train_final = vectorizer.fit_transform(X_train)
X_test_final = vectorizer.transform(X_test)
print(f'X_train_final shape: {X_train_final.shape}')
print(f'X_test_final shape: {X_test_final.shape}')

model = LogisticRegression(max_iter=200, C=10, random_state=RANDOM_STATE, n_jobs=-1)
model.fit(X_train_final, y_train)

predicted = model.predict(X_test_final)
print(f'F1 на тестовой выборке: {f1_score(y_test, predicted)}')

X_train_final shape: (119469, 134807)
X_test_final shape: (39823, 134807)
F1 на тестовой выборке: 0.7713103448275863


Модель подтвердила высокое качество на тестовой выборке.

## Выводы

Мы обработали данные, очистили текст комментария от лишних символов.

Затем разбили текст на слова и привели каждое слово к его начальной форме.

После этого выполнили векторизацию текстов и на основе полученных векторов провели обучение моделей.

Наилучшей моделью оказалась **LogisticRegression**